## multi-class kernel perceptron using the One-vs-One strategy

In [43]:
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
from scipy.spatial.distance import cdist


In [44]:
# Useful functions
k=10

def compute_polynomial_kernel(P, Q, d=3):
    # dot product between pat and each row in dat
    if P.ndim == 1:
        P = P.reshape(1, -1)
    K = (np.dot(P, Q.T)) ** d
    return K

def compute_gaussian_kernel(P, Q, c):
    if P.ndim == 1:
        P = P.reshape(1, -1)
    # distance = np.sum((P[:, np.newaxis, :] - Q)**2, axis=2)
    distance_sq = cdist(P, Q, 'sqeuclidean')
    return np.exp(-c * distance_sq)

def mysign(x):
    return np.where(x <= 0.0, -1.0, 1.0)
    # return -1.0 if x <= 0.0 else 1.0

def get_train_test(df,split_proportion=0.8):
    """
     Splits data into train and test sets based on split_proportion.
     Args:
     	 df: pandas dataframe to be split
     	 split_proportion: proportion of all samples to be used for training

     Returns:
     	 trainX, trainY, testX, testY
    """
    shuffledidx = np.random.permutation(df.index)
    m = len(df.index)
    nsamples_train = int(split_proportion*m)
    train = df.iloc[shuffledidx[:nsamples_train]].to_numpy()
    test = df.iloc[shuffledidx[nsamples_train:]].to_numpy()

    return train, test

In [ ]:
class OneVsOneKclassPerceptron:
    def __init__(self,kclasses,debug=False):
        """
        Initializes the model with k classes and builds a list of binary classifier pairs: 
        for k classes, this results in k(k-1)/2 binary classifiers.
        """
        self.kclasses = kclasses #len(class_labels)
        self.train_fixed_epochs = True
        self.debug = debug
        self.showComputationTime = False
        # self.classes = np.unique(class_labels)

        self.num_classifiers = int(self.kclasses * (self.kclasses-1) /2)  # No of binary classifiers
        # generate pairs of unique classes
        self.class_combinations = [[i, j] for i in range(k) for j in range(i+1, k)]
        self.classifiers = self.class_combinations #  map pairwise combination of classes to label



    def predict(self, K_test):
        """
        Predicts the class label for a given test sample based on the kernel matrix and classifier weights.
        Method:
        1. Computes votes from all binary classifiers.
        2. Each classifier votes for one of two classes depending on the sign of the kernel-based prediction.
        3. The final prediction is the class with the most votes.
        K_test: kernel matrix for the test sample
        alphas: weights for each binary classifier
        Returns:
            y_pred: predicted class label for the test sample
        """
        # np.sign returns -1, 0, or 1 based on the sign of the dot product
        votes = np.sign( np.dot(self.alphas,K_test)) # votes from each binary classifier
        total_votes_each_class = np.zeros(self.kclasses)
        for i in range(self.num_classifiers):  
            vote = self.classifiers[i][0] if votes[i] > 0 else self.classifiers[i][1]  # get class vote of classifier
            total_votes_each_class[int(vote)] += 1 
        y_pred = np.argmax(total_votes_each_class) #max voting
        return y_pred


    def fit_online(self,trainX, trainY, kernel_matrix_train, n_epochs, poly_dim=None):
        """
        Trains the perceptron using the precomputed kernel matrix in an online manner for n_epochs.
        Args:
            trainX: training data features
            trainY: training data labels
            kernel_matrix_train: precomputed kernel matrix for the training set
            n_epochs: number of epochs to train the model
            poly_dim: polynomial degree for the kernel (if using polynomial kernel)
        """        
        self.trainX = trainX
        self.trainY = trainY
        self.K_train = kernel_matrix_train
        self.len_trainset = len(self.trainY)
        self.poly_dim = poly_dim
        
        self.alphas = np.zeros((self.num_classifiers, self.len_trainset)) #alphas - classifier weights

        for current_epoch in range(1,n_epochs+1):
            # Training
            self.trainOneEpoch()
            train_error = round(self.mistakes/self.len_trainset,4)
            print(f'---->Epoch {current_epoch}, train_error: {train_error} ')
        return train_error

    def trainOneEpoch(self):
        '''Iterates over all training examples, uses the kernel matrix to predict, and updates the corresponding alpha values if there's a mistake'''
        mistakes = 0
        for i in range(self.len_trainset):
            current_kernel_matrix = self.K_train[i].flatten() # transformation of current input features
            prediction = self.predict(current_kernel_matrix)
            current_label = self.trainY[i]
            
            if prediction != current_label:
                mistakes+=1
                # looping every classifier which involves true label - index of alpha parameters for each combination
                for j in range(self.num_classifiers):   
                    if current_label in self.classifiers[j]:
                        self.alphas[j,i]+=1 if self.classifiers[j][0]==current_label else -1
        self.mistakes = mistakes

    def test(self,testX,testY):
        mistakes = 0
        len_testset = len(testY)
        if len_testset == 1:
            testX = testX.reshape(1, -1)
        test_kernel_matrix = compute_polynomial_kernel(testX,self.trainX, self.poly_dim)
        preds = []
        for i in range(len_testset):
            true_y = testY[i]
            current_kernel_matrix = test_kernel_matrix[i].flatten()
            prediction = self.predict(current_kernel_matrix)
            preds.append(prediction)
            if prediction != true_y:
                mistakes+=1
        test_error = mistakes / len_testset
        print(f'---> Test_error: {test_error} ')
        return preds,test_error


In [46]:
dataset_alldigits = pd.read_csv('zipcombo.dat',header=None,delim_whitespace=True)
polydim_values = np.arange(1,8)

In [47]:
# Code for Q1

train_error_polydim = {}
test_error_polydim = {}
for idx in polydim_values:
    train_error_polydim[idx] = []
    test_error_polydim[idx] = []

start_time = time.time()
n_epochs = 9 # optimized by taking median of indexes giving best validation error
for repeat in range(20): #TODO: 20

    print(f'Run {repeat+1}/20: ')
    # Split data into train and test
    train_data, test_data = get_train_test(dataset_alldigits,split_proportion=0.8)
    trainX, trainY = train_data[:, 1:], train_data[:, 0]
    testX, testY = test_data[:, 1:], test_data[:, 0]
    # print(len(test_data), len(test_data), trainY[0], testY[0] )

    for poly_dim in polydim_values:
        print(f'poly_dim: {poly_dim}')
        K_train = compute_polynomial_kernel(trainX, trainX, poly_dim)

        model =     OneVsOneKclassPerceptron(kclasses=10)
        train_error = model.fit_online(trainX= trainX, trainY = trainY, kernel_matrix_train = K_train, n_epochs=n_epochs, poly_dim=poly_dim)
        _,test_error = model.test(testX, testY)

        train_error_polydim[poly_dim].append(train_error)
        test_error_polydim[poly_dim].append(test_error)

testing_time = time.time() - start_time
print(f"Algorithm took {testing_time:.2f} seconds")

Run 1/20: 
poly_dim: 1
---->Epoch 1, train_error: 0.1379 
---->Epoch 2, train_error: 0.1023 
---->Epoch 3, train_error: 0.0953 
---->Epoch 4, train_error: 0.094 
---->Epoch 5, train_error: 0.0913 
---->Epoch 6, train_error: 0.0921 
---->Epoch 7, train_error: 0.0887 
---->Epoch 8, train_error: 0.0881 
---->Epoch 9, train_error: 0.0869 
---> Test_error: 0.10268817204301076 
poly_dim: 2
---->Epoch 1, train_error: 0.1015 
---->Epoch 2, train_error: 0.0664 
---->Epoch 3, train_error: 0.0566 
---->Epoch 4, train_error: 0.05 
---->Epoch 5, train_error: 0.0479 
---->Epoch 6, train_error: 0.0465 
---->Epoch 7, train_error: 0.0428 
---->Epoch 8, train_error: 0.039 
---->Epoch 9, train_error: 0.0417 
---> Test_error: 0.07204301075268817 
poly_dim: 3
---->Epoch 1, train_error: 0.0917 
---->Epoch 2, train_error: 0.0391 
---->Epoch 3, train_error: 0.0281 
---->Epoch 4, train_error: 0.0231 
---->Epoch 5, train_error: 0.0194 
---->Epoch 6, train_error: 0.0151 
---->Epoch 7, train_error: 0.0136 
---->E

In [48]:
# 1. Answers
mean_train_error, mean_test_error, sd_train_error, sd_test_error = {}, {}, {}, {}

for key, values in train_error_polydim.items():
    mean_train_error[key] = np.mean(values)
    sd_train_error[key] = np.std(values)
for key, values in test_error_polydim.items():
    mean_test_error[key] = np.mean(values)
    sd_test_error[key] = np.std(values)

print('Mean of Train errors for each d \n ',', '.join(f'{key}: {value:.6f}' for key, value in mean_train_error.items()))
print('SD of Train errors for each d \n ',', '.join(f'{key}: {value:.6f}' for key, value in sd_train_error.items()))
print('Mean of Test errors for each d \n ',', '.join(f'{key}: {value:.6f}' for key, value in mean_test_error.items()))
print('SD of Test errors for each d \n ',', '.join(f'{key}: {value:.6f}' for key, value in sd_test_error.items()))

Mean of Train errors for each d 
  1: 0.090255, 2: 0.040300, 3: 0.013625, 4: 0.008175, 5: 0.004650, 6: 0.003265, 7: 0.002630
SD of Train errors for each d 
  1: 0.002473, 2: 0.002608, 3: 0.001888, 4: 0.001987, 5: 0.001660, 6: 0.001397, 7: 0.001576
Mean of Test errors for each d 
  1: 0.102339, 2: 0.069677, 3: 0.058548, 4: 0.055430, 5: 0.053280, 6: 0.051909, 7: 0.051075
SD of Test errors for each d 
  1: 0.007821, 2: 0.005888, 3: 0.004581, 4: 0.005469, 5: 0.005131, 6: 0.005105, 7: 0.005360


In [49]:
polydim_values = np.arange(1,8)

best_20_d = [] # best 20 of poly_dim in crossvalidation
test_errors_chosen_params = []


start_time = time.time()
n_epochs = 5 #: optimized above


for run_index in range(20): #TODO:20
    print(f'Run {run_index+1}/20: ')
    # Split data into train and test
    big_train_data, final_test_data = get_train_test(dataset_alldigits, split_proportion=0.8)

    big_train_dataX, big_train_dataY = big_train_data[:, 1:], big_train_data[:, 0]
    final_test_dataX, final_test_dataY = final_test_data[:, 1:], final_test_data[:, 0]

    # Create folds for cross validation
    n_folds = 5
    fold_size = int(len(big_train_data)/n_folds)
    fold_indices = []
    indices = np.arange(len(big_train_data))
    for i in range(n_folds):
        start_idx, end_idx = i * fold_size,  (i + 1) * fold_size
        validation_indices = indices[start_idx: end_idx]
        train_indices = np.concatenate([indices[:start_idx], indices[end_idx:]])
        fold_indices.append((train_indices, validation_indices))


    chosen_polydim = None
    best_score = float('inf')
    cv_errors = np.zeros(len(polydim_values))

    print('--Iterating over poly_dims')
    for poly_dim_iter in polydim_values:
        print(f'-->poly_dim: {poly_dim_iter}')
        fold_scores = []
        for fold_num, (train_indices, validation_indices) in enumerate(fold_indices):
            print("--->Fold ",fold_num)
            train_foldX, validation_foldX = big_train_dataX[train_indices], big_train_dataX[validation_indices]
            train_foldY, validation_foldY = big_train_dataY[train_indices], big_train_dataY[validation_indices]

            # Train the model on the training data and make predictions on the validation data
            K_trainfold = compute_polynomial_kernel(train_foldX, train_foldX, poly_dim_iter)
            model =     OneVsOneKclassPerceptron(kclasses=10)

            train_error = model.fit_online(trainX= train_foldX, trainY = train_foldY, kernel_matrix_train = K_trainfold, n_epochs=n_epochs, poly_dim=poly_dim_iter)
            _,validation_error = model.test(validation_foldX, validation_foldY)
            # print(train_error, validation_error)
            fold_scores.append(validation_error)

        average_score = np.mean(fold_scores)
        if average_score < best_score:
            best_score = average_score
            chosen_polydim = poly_dim_iter
    print('** Best d for the run :',chosen_polydim, 'with average test error:',best_score)
    best_20_d.append(chosen_polydim)

    # Retrain the model with best d on the entire training set and get error on test set
    K_big_train_data = compute_polynomial_kernel(big_train_dataX, big_train_dataX, chosen_polydim)
    model =     OneVsOneKclassPerceptron(kclasses=10)
    train_error = model.fit_online(trainX= big_train_dataX, trainY = big_train_dataY, kernel_matrix_train = K_big_train_data, n_epochs=n_epochs, poly_dim=chosen_polydim)
    predictions_currentrun , test_error = model.test(final_test_dataX, final_test_dataY)

    # for q2
    print('--> Test set error',test_error)
    test_errors_chosen_params.append(test_error)


crossval_time = time.time() - start_time
print(f"Algorithm took {crossval_time:.2f} seconds")


Run 1/20: 
--Iterating over poly_dims
-->poly_dim: 1
--->Fold  0
---->Epoch 1, train_error: 0.1544 
---->Epoch 2, train_error: 0.1091 
---->Epoch 3, train_error: 0.1042 
---->Epoch 4, train_error: 0.0998 
---->Epoch 5, train_error: 0.0995 
---> Test_error: 0.1062542030934768 
--->Fold  1
---->Epoch 1, train_error: 0.1583 
---->Epoch 2, train_error: 0.1138 
---->Epoch 3, train_error: 0.1055 
---->Epoch 4, train_error: 0.1038 
---->Epoch 5, train_error: 0.0995 
---> Test_error: 0.09213180901143242 
--->Fold  2
---->Epoch 1, train_error: 0.1524 
---->Epoch 2, train_error: 0.1091 
---->Epoch 3, train_error: 0.0998 
---->Epoch 4, train_error: 0.0939 
---->Epoch 5, train_error: 0.0904 
---> Test_error: 0.10221923335574983 
--->Fold  3
---->Epoch 1, train_error: 0.1522 
---->Epoch 2, train_error: 0.1049 
---->Epoch 3, train_error: 0.0971 
---->Epoch 4, train_error: 0.0951 
---->Epoch 5, train_error: 0.0941 
---> Test_error: 0.11028917283120376 
--->Fold  4
---->Epoch 1, train_error: 0.1511 
-

In [50]:
# Answers for q2
print(best_20_d)
test_errors_chosen_params = np.array(test_errors_chosen_params)
print(test_errors_chosen_params, test_errors_chosen_params.mean(), test_errors_chosen_params.std())

[7, 7, 5, 7, 5, 7, 7, 6, 7, 6, 7, 6, 6, 7, 7, 7, 7, 7, 7, 7]
[0.04946237 0.0516129  0.04193548 0.0516129  0.05698925 0.04731183
 0.04946237 0.04139785 0.0516129  0.04354839 0.04946237 0.04569892
 0.04731183 0.05       0.05053763 0.05       0.03655914 0.05268817
 0.04408602 0.05      ] 0.04806451612903227 0.004579054698896255


In [2]:
best_20_d = [7, 7, 5, 7, 5, 7, 7, 6, 7, 6, 7, 6, 6, 7, 7, 7, 7, 7, 7, 7]
np.mean(best_20_d), np.std(best_20_d)

(6.6, 0.6633249580710799)